In [46]:
# 라이브러리 기본 참조
from hossam import load_data
from pandas import DataFrame, concat, pivot_table, get_dummies
from matplotlib import pyplot as plt
import pandas as pd
from matplotlib import font_manager as fm
import seaborn as sb
import numpy as np
from scipy.stats import t, ttest_ind, normaltest, bartlett, levene
from math import sqrt


# 선형회귀를 위한 참조
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 결과 검정을 위한 참조
from statsmodels.stats.diagnostic import linear_reset
from scipy.stats import zscore, probplot, shapiro, anderson, pearsonr, spearmanr
from statsmodels.stats.api import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
from pingouin import anova, welch_anova, pairwise_tukey, pairwise_gameshowell

# VIF값을 계산하기 위한 참조
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statannotations.Annotator import Annotator

# 파이썬 기본 패키지
from itertools import combinations

In [47]:
origin = load_data("diamonds")
print("\n===== 데이터 크기 확인 =====")
print(f"데이터셋 크기: {origin.shape}")
print(f"열 개수: {origin.shape[1]}")
print(f"행 개수: {origin.shape[0]}")
print("\n===== 타입확인 =====")
print(origin.info())
origin.head()

[data] https://data.hossam.kr/data/kaggle/diamonds.xlsx
[desc] 다이아몬드를 커팅, 색상, 투명도, 가격 및 기타 속성 조사한 데이터 셋 (출처: https://www.kaggle.com/datasets/shivam2503/diamonds)

field    description
-------  -------------------------------------------------------------------
price    다이아몬드 가격 (USD, $326 ~ $18,823)
carat    중량 (0.2~5.01)
cut      컷 품질 (Fair, Good, Very Good, Premium, Ideal)
color    색상 등급 - J (worst) to D (best)
clarity  투명도 등급 (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))
x        길이 mm (0~10.74)
y        너비 mm (0--58.9)
z        두께 mm (0--31.8)
depth    비율 정보 = z / mean(x, y) = 2 * z / (x + y) (43--79)
table    다이아몬드의 가장 넓은 지점에 비해 상단(테이블)의 너비 (43--95)


===== 데이터 크기 확인 =====
데이터셋 크기: (53940, 10)
열 개수: 10
행 개수: 53940

===== 타입확인 =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   price    53940 non-null  int64  
 1   carat    53940 

,price,carat,cut,color,clarity,x,y,z,depth,table
0,326,0.23,Ideal,E,SI2,3.95,3.98,2.43,61.5,55.0
1,326,0.21,Premium,E,SI1,3.89,3.84,2.31,59.8,61.0
2,327,0.23,Good,E,VS1,4.05,4.07,2.31,56.9,65.0
3,334,0.29,Premium,I,VS2,4.20,4.23,2.63,62.4,58.0
4,335,0.31,Good,J,SI2,4.34,4.35,2.75,63.3,58.0


In [48]:
def ols_report(tbl, data):
    """
    OLS summary(tbl)와 데이터프레임(data)을 받아 회귀계수 표(rdf),
    모델 적합도 요약(result_report), 모형 보고 문장(model_report),
    변수별 보고 리스트(variable_reports)를 반환한다.

    Parameters
    ----------
    tbl : statsmodels.iolib.summary.Summary
        sm.OLS(...).fit().summary() 결과로 생성된 summary 객체.
    data : pandas.DataFrame
        종속변수와 독립변수를 모두 포함한 데이터프레임.

    Returns
    -------
    tuple
        (회귀계수 DataFrame, 적합도 요약 문자열, 모형 보고 문자열, 변수별 보고 리스트)
    """

    # 독립변수 이름(상수항 제외)
    xnames = [n for n in fit.model.exog_names if n != "const"]

    # 독립변수 부분 데이터 (VIF 계산용)
    indi_df = data.filter(xnames)

    # 독립변수 결과를 누적
    variables = []

    for i, v in enumerate(tbl.tables[1].data):
        # 한 행의 변수명 추출 후 목록에 있는지 확인
        name = v[0].strip()
        if name not in xnames:
            continue

        # VIF 계산을 위해 열 인덱스 조회
        j = list(indi_df.columns).index(name)
        vif = variance_inflation_factor(indi_df, j)

        # 유의확률과 별표 표시 함수
        p = float(v[4].strip())

        def stars(p):
            return "***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else ""

        # 한 변수에 대한 보고 정보 추가
        variables.append(
            {
                "종속변수": yname,  # 종속변수 이름 (외부에서 정의)
                "독립변수": name,  # 독립변수 이름
                "B": v[1].strip(),  # 비표준화 회귀계수(B)
                "표준오차": v[2].strip(),  # 계수 표준오차
                "Beta": float(fit.params[name])
                * (
                    data[name].std(ddof=1) / data[yname].std(ddof=1)
                ),  # 표준화 회귀계수(β)
                "t": "%s%s" % (v[3].strip(), stars(p)),  # t-통계량(+별표)
                "p-value": p,  # 계수 유의확률
                "공차": 1 / vif,  # 공차(Tolerance = 1/VIF)
                "vif": vif,  # 분산팽창계수
            }
        )

    rdf = DataFrame(variables)

    # summary 표에서 적합도 정보를 key-value로 추출
    result_dict = {}
    for idx in [0, 2]:
        for item in tbl.tables[idx].data:
            n = len(item)
            for j in range(0, n, 2):
                key = item[j].strip()[:-1]
                value = item[j + 1].strip()
                if not key or not value:
                    continue
                result_dict[key] = value

    # 적합도 보고 문자열 구성
    result_report = (
        f"𝑅({result_dict['R-squared']}), "
        f"𝑅^2({result_dict['Adj. R-squared']}), "
        f"𝐹({result_dict['F-statistic']}), "
        f"유의확률({result_dict['Prob (F-statistic)']}), "
        f"Durbin-Watson({result_dict['Durbin-Watson']})"
    )

    # 모형 보고 문장 구성
    p_f = float(result_dict["Prob (F-statistic)"])
    sig_text = "유의하다" if p_f <= 0.05 else "유의하지 않다"
    p_sign = "≤" if p_f <= 0.05 else ">"

    tpl = (
        "%s에 대하여 %s로 예측하는 회귀분석을 실시한 결과, "
        "이 회귀모형은 통계적으로 %s(F(%s,%s) = %s, p %s 0.05)."
    )

    model_report = tpl % (
        rdf["종속변수"][0],
        ",".join(list(rdf["독립변수"])),
        sig_text,
        result_dict["Df Model"],
        result_dict["Df Residuals"],
        result_dict["F-statistic"],
        p_sign,
    )

    # 변수별 보고 문장 리스트 구성
    variable_reports = []
    s = "%s의 회귀계수는 %s(p %s 0.05)로, %s에 대하여 %s 예측변인인 것으로 나타났다."

    for i in rdf.index:
        row = rdf.iloc[i]
        p_val = float(row["p-value"])
        p_sign_var = "≤" if p_val < 0.05 else ">"
        sig_word = "유의미한" if p_val < 0.05 else "유의하지 않은"

        variable_reports.append(
            s
            % (
                row["독립변수"],
                row["B"],
                p_sign_var,
                row["종속변수"],
                sig_word,
            )
        )

    return rdf, result_report, model_report, variable_reports

In [49]:
xyz = ["x", "y", "z"]
origin.drop_duplicates()
df = origin[(origin[xyz] != 0).all(axis=1)]

df

,price,carat,cut,color,clarity,x,y,z,depth,table
0,326,0.23,Ideal,E,SI2,3.95,3.98,2.43,61.5,55.0
1,326,0.21,Premium,E,SI1,3.89,3.84,2.31,59.8,61.0
2,327,0.23,Good,E,VS1,4.05,4.07,2.31,56.9,65.0
3,334,0.29,Premium,I,VS2,4.20,4.23,2.63,62.4,58.0
4,335,0.31,Good,J,SI2,4.34,4.35,2.75,63.3,58.0
...,...,...,...,...,...,...,...,...,...,...
53935,2757,0.72,Ideal,D,SI1,5.75,5.76,3.50,60.8,57.0
53936,2757,0.72,Good,D,SI1,5.69,5.75,3.61,63.1,55.0
53937,2757,0.70,Very Good,D,SI1,5.66,5.68,3.56,62.8,60.0
53938,2757,0.86,Premium,H,SI2,6.15,6.12,3.74,61.0,58.0


In [50]:
df = df.copy()

df["log_price"] = np.log(df["price"])
df["log_carat"] = np.log(df["carat"])
df.head()

,price,carat,cut,color,clarity,x,y,z,depth,table,log_price,log_carat
0,326,0.23,Ideal,E,SI2,3.95,3.98,2.43,61.5,55.0,5.786897,-1.469676
1,326,0.21,Premium,E,SI1,3.89,3.84,2.31,59.8,61.0,5.786897,-1.560648
2,327,0.23,Good,E,VS1,4.05,4.07,2.31,56.9,65.0,5.789960,-1.469676
3,334,0.29,Premium,I,VS2,4.20,4.23,2.63,62.4,58.0,5.811141,-1.237874
4,335,0.31,Good,J,SI2,4.34,4.35,2.75,63.3,58.0,5.814131,-1.171183


In [51]:
df1 = df.filter(["log_price", "log_carat", "depth", "table", "x", "y", "z"]).copy()
df1.head()

yname = "log_price"

x = df1.drop(yname, axis=1)
y = df1[yname]

x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()
print(tbl1)

                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.935
Model:                            OLS   Adj. R-squared:                  0.935
Method:                 Least Squares   F-statistic:                 1.299e+05
Date:                Thu, 15 Jan 2026   Prob (F-statistic):               0.00
Time:                        12:08:54   Log-Likelihood:                -3473.7
No. Observations:               53920   AIC:                             6961.
Df Residuals:                   53913   BIC:                             7024.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         11.5899      0.116     99.541      0.0

In [52]:
rdf, r, m, v = ols_report(tbl1, df1)
display(rdf)
print(m)

,종속변수,독립변수,B,표준오차,Beta,t,p-value,공차,vif
0,log_price,log_carat,1.7378,0.018,1.001587,95.637***,0.000,0.019411,51.516387
1,log_price,depth,-0.0309,0.001,-0.043646,-29.583***,0.000,0.000982,1017.838106
2,log_price,table,-0.0186,0.001,-0.040992,-34.684***,0.000,0.001683,594.130693
3,log_price,x,-0.0424,0.011,-0.046778,-3.842***,0.000,0.000645,1549.435070
4,log_price,y,0.0091,0.004,0.010224,2.049*,0.040,0.001850,540.537766
5,log_price,z,0.0149,0.009,0.010311,1.719,0.086,0.001329,752.453780


log_price에 대하여 log_carat,depth,table,x,y,z로 예측하는 회귀분석을 실시한 결과, 이 회귀모형은 통계적으로 유의하다(F(6,53913) = 1.299e+05, p ≤ 0.05).


In [53]:
while True:
    rdf, r, m, v = ols_report(tbl1, df1)
    if rdf["vif"].max() >= 10:
        df1 = df1.drop(rdf[rdf["vif"] == rdf["vif"].max()]["독립변수"], axis=1)
        x = df1.drop(yname, axis=1)
        y = df1[yname]
        x_input = sm.add_constant(x)
        fit = sm.OLS(y, x_input).fit()
        tbl1 = fit.summary()
    else:
        break

In [54]:
display(rdf)

,종속변수,독립변수,B,표준오차,Beta,t,p-value,공차,vif
0,log_price,log_carat,1.7161,0.009,0.989099,192.725***,0.0,0.846452,1.181402
1,log_price,z,-0.0343,0.007,-0.023744,-4.627***,0.0,0.846452,1.181402


In [55]:
df2 = df.filter(["log_price", "log_carat", "cut", "color", "clarity"]).copy()

df2 = get_dummies(
    df2, columns=["cut", "color", "clarity"], dtype="int", drop_first=True
)
df2.head()

,log_price,log_carat,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_E,color_F,color_G,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,5.786897,-1.469676,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
1,5.786897,-1.560648,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0
2,5.789960,-1.469676,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,5.811141,-1.237874,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0
4,5.814131,-1.171183,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0


In [56]:
yname = "log_price"

x = df2.drop(yname, axis=1)
y = df2[yname]

x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()
print(tbl1)

                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.983
Model:                            OLS   Adj. R-squared:                  0.983
Method:                 Least Squares   F-statistic:                 1.693e+05
Date:                Thu, 15 Jan 2026   Prob (F-statistic):               0.00
Time:                        12:08:54   Log-Likelihood:                 31961.
No. Observations:               53920   AIC:                        -6.388e+04
Df Residuals:                   53901   BIC:                        -6.372e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             7.8568      0.006   1362.891

In [57]:
rdf, r, m, v = ols_report(tbl1, df2)
display(rdf)
print(m)

,종속변수,독립변수,B,표준오차,Beta,t,p-value,공차,vif
0,log_price,log_carat,1.8837,0.001,1.085710,1668.528***,0.0,0.522840,1.912630
1,log_price,cut_Good,0.0801,0.004,0.022708,20.592***,0.0,0.284892,3.510103
2,log_price,cut_Ideal,0.1613,0.004,0.077862,45.442***,0.0,0.080128,12.480108
3,log_price,cut_Premium,0.1394,0.004,0.059934,38.942***,0.0,0.122948,8.133493
4,log_price,cut_Very Good,0.1172,0.004,0.048185,32.392***,0.0,0.135808,7.363356
5,log_price,color_E,-0.0542,0.002,-0.020610,-25.600***,0.0,0.429124,2.330329
6,log_price,color_F,-0.0945,0.002,-0.035560,-44.135***,0.0,0.433958,2.304371
7,log_price,color_G,-0.1602,0.002,-0.064237,-76.405***,0.0,0.383126,2.610105
8,log_price,color_H,-0.2511,0.002,-0.089295,-112.834***,0.0,0.462241,2.163375
9,log_price,color_I,-0.3725,0.002,-0.110417,-149.483***,0.0,0.555656,1.799675


log_price에 대하여 log_carat,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_E,color_F,color_G,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2로 예측하는 회귀분석을 실시한 결과, 이 회귀모형은 통계적으로 유의하다(F(18,53901) = 1.693e+05, p ≤ 0.05).


In [58]:
xnames = list(rdf[rdf["p-value"] > 0.05]["독립변수"])
df3 = df2.drop(xnames, axis=1)
x = df3.drop(yname, axis=1)
y = df3[yname]
x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl = fit.summary()
rdf, r, m, v = ols_report(tbl, df3)

In [59]:
r

'𝑅(0.983), 𝑅^2(0.983), 𝐹(1.693e+05), 유의확률(0.00), Durbin-Watson(1.245)'